# Covid-19 Vaccines in California and their Impacts

## Data Cleaning and Exploration

In [ ]:
import pandas as pd
import scipy.stats as sts
import matplotlib.pyplot as plt
import datetime as dt
import requests
from census import Census
from us import states
import json

#import API keys
from config import api_key
from config import gkey
c = Census(api_key)

### Cleaning Data

In [ ]:
# Cleaning Cases Data
case_df = pd.read_csv("Data/covid19cases_test.csv")

# change date column to easier to compare format
case_df.date = pd.to_datetime(case_df['date'],format='%m/%d/%Y')

# filter out dates past 3-31-21, as well as rows with useless data
clean_case = case_df.loc[(case_df.date<dt.datetime(2021,4,1)) & \
                         (case_df.area != 'Out of state') & \
                         (case_df.area != 'California')& \
                         (case_df.area != 'Unknown')]

# filter out columns we dont need
clean_case = clean_case[['date','area','population','cases','deaths','total_tests','positive_tests']]

# Rename column for dataset consistency
clean_case = clean_case.rename(columns={'area': 'county'})

# replace missing data with 0
clean_case = clean_case.fillna(0)
clean_case.to_csv('Data/clean_case.csv', index=False)

clean_case.head()

In [ ]:
# Cleaning Vaccinations Data
vaccine_df = pd.read_csv("Data/covid19vaccinesbycounty.csv")
vaccine_df.head()

# Dropping unneeded columns
vaccine_df=vaccine_df.drop(columns=['cumulative_pfizer_doses',
                         'cumulative_moderna_doses','cumulative_jj_doses',
                         'cumulative_at_least_one_dose', 'california_flag'])
vaccine_df=vaccine_df.drop(vaccine_df[(vaccine_df['county']=='All CA Counties')|
                                      (vaccine_df['county']=='All CA and Non-CA Counties')|
                                      (vaccine_df['county']=='Outside California')|
                                      (vaccine_df['county']=='Unknown')].index)

vaccine_df.to_csv('Data/vaccine_df.csv', index=False)

vaccine_df.head()

In [ ]:
# Cleaning Hospital Data

In [ ]:
# Cleaning Colleges Data
college_data = pd.read_csv('Data/ca_colleges.csv')

college_data.columns = ['School_ID', 'School_name', 'Year', 'Duplicate', 'Street_Address', 'City', 'Zip_code', 
                        'county', 'Longitude', 'Latitude', 'Total_Enrollment', 'Full_time_enrollment', 
                        'Part_time_enrollment']

college_data.drop(columns=['School_ID', 'Year', 'Duplicate', 'Street_Address', 'City', 'Zip_code', 
                        'Full_time_enrollment', 'Part_time_enrollment'], inplace=True)

college_data.county = college_data.county.map(lambda x: x[ :-7])
college_data.dropna(subset=['Total_Enrollment'], inplace=True)
college_data.to_csv('Data/college_data.csv', index=False)

college_data.head()

In [ ]:
#Determining California fips code for API
states.CA.fips

# Obtaining US Census Data
census_data = c.acs5.get(('NAME', 'B19013_001E', 'B19301_001E', 'B17001_002E', 
                         'B01003_001E'), geo={'for': 'zip code tabulation area:*', 'in': 'state:06'})
census_data[0]

#Converting to DataFrame
census_df = pd.DataFrame(census_data)

census_df.rename(columns={'B19013_001E': 'Household_Income',
                         'B19301_001E': 'Income_per_Capita',
                         'B17001_002E': 'Poverty_Count', 
                         'B01003_001E': 'Population', 
                         'NAME': 'Name', 
                          'zip code tabulation area': 'Zipcode'}, inplace=True)
census_df = census_df.loc[census_df.Household_Income>0].copy()
census_df.drop(columns=['Name', 'Poverty_Count', 'state'], inplace=True)
census_df = census_df[['Zipcode', 'Household_Income', 'Income_per_Capita', 'Population']]

census_df.to_csv('Data/census_df.csv', index=False)
census_df.head()

### Data Grouping

In [ ]:
#Vaccines by Date:
ca_vaccinations=vaccine_df.groupby(['administered_date'],as_index=False).agg({'cumulative_total_doses':['sum']})
ca_vaccinations.columns = ca_vaccinations.columns.droplevel(1)
ca_vaccinations.administered_date = pd.to_datetime(ca_vaccinations['administered_date'],format='%m/%d/%Y')
ca_vaccinations = ca_vaccinations.loc[(ca_vaccinations.administered_date<dt.datetime(2021,4,1))]
ca_vaccinations['cumulative_doses_by_10000']=ca_vaccinations.apply(lambda row: row.cumulative_total_doses/10000,axis=1)

ca_vaccinations.to_csv('Data/ca_vaccinations.csv', index=False)
ca_vaccinations

In [ ]:
# Cases by County:
cases_grouped = clean_case.groupby('county').agg({'cases': 'sum',
                                                  'deaths': 'sum',
                                                  'total_tests': 'sum',
                                                  'positive_tests': 'sum',
                                                  'population': 'max'})

cases_grouped['Cases per 100,000'] = (cases_grouped['cases']/cases_grouped['population']*100000).astype(int)

cases_grouped.info()

In [ ]:
# Vaccines by County:
vaccine_grouped = vaccine_df.groupby('county').agg({'total_doses': 'sum',
                                                  'cumulative_fully_vaccinated': 'max'})

vaccine_grouped['Fully Vaccinated per 100,000'] = (vaccine_grouped['cumulative_fully_vaccinated']/
                                                         cases_grouped['population']*100000).astype(int)

vaccine_grouped.info()

Both cases_grouped and vaccines_grouped dataframes contain all 58 counties.

In [ ]:
grouped_colleges = college_data.groupby('county')['Total_Enrollment'].sum().astype(int)
college_population = grouped_colleges.to_frame()

len(college_population)

41 of California's counties contain at least one college with student enrollment for the 2019-2020 school year.

#### Running geocodng API to obtain lat/lon values for counties

In [ ]:
# Create a list of Counties included in data sets to use for geocoding API

counties = clean_case['county'].unique()

# Creating Lists to fill coordinates
lat = []
lng = []

# Set up for API call
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='
    
for county in counties:
    target_county = f'{base_url}{county},+CA&key={gkey}'
    response = requests.get(target_county).json()
    try:
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lng.append(response['results'][0]['geometry']['location']['lng'])
    except:
        print(f'{county} could not be located...Skipping')
        pass
print('Coordinates have been added to datasets.')

In [ ]:
# Build county DF

county_dict = {'county': counties,
              'latitude': lat,
              'longitude': lng}
county_data = pd.DataFrame(county_dict)

county_data.set_index('county', inplace=True)
county_data['population'] = cases_grouped['population']
county_data['student_pop'] = college_population['Total_Enrollment']
county_data['cases per 100,000'] = cases_grouped['Cases per 100,000']
county_data['Fully Vaccinated per 100,000'] = vaccine_grouped['Fully Vaccinated per 100,000']
county_data.info()

In [ ]:
county_data['student_pop'] = county_data['student_pop'].fillna(0)
county_data.reset_index(inplace=True)
county_data['% student population'] = round(county_data['student_pop']/county_data['population']*100,2)

county_data.to_csv('Data/county_data.csv', index=False)
county_data.head()